In [ ]:
import requests
import pandas as pd
import time
import string

APP_ID = "5b720389"
API_KEY = "0a74f60c5c4ebeed8ed3c975f1844fd3"
url = "https://trackapi.nutritionix.com/v2/search/instant"
headers = {
    "x-app-id": APP_ID,
    "x-app-key": API_KEY,
    "Content-Type": "application/json"
}

results = []
food_names_set = set()

search_terms = list(string.ascii_lowercase)

for term in search_terms:
    params = {
        "query": term,
        "branded": False,
        "common": True,
        "self": False
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Error {response.status_code} for query '{term}': {response.text}")
        continue

    data = response.json()
    common_foods = data.get("common", [])
    for food in common_foods:
        fname = food.get("food_name")
        if fname and fname not in food_names_set:
            food_names_set.add(fname)
            results.append({
                "food_name": fname,
                "serving_unit": food.get("serving_unit"),
            })
        if len(results) >= 200:
            break

    if len(results) >= 200:
        break

    time.sleep(0.5)

df = pd.DataFrame(results)
df.to_csv("nutritio_data.csv", index=False)
print(f"Fetched {len(results)} unique common foods and saved to nutritionix_unique_foods.csv")